In [1]:
import os, dotenv
dotenv.load_dotenv()

from src.cira import CiRAConverter

The pre-trained classification and labeling model must be available on your local machine. Both models can be found on Zenodo:

* Classifier: [https://zenodo.org/record/5159501#.Ytq28ITP3-g](https://zenodo.org/record/5159501#.Ytq28ITP3-g)
* Labeler (use the model named roberta_dropout_linear_layer_multilabel.ckpt for optimal performance): [https://zenodo.org/record/5550387#.Ytq3QYTP3-g](https://zenodo.org/record/5550387#.Ytq3QYTP3-g) 

In [2]:
# specify the location of the pre-trained classification and labeling model
model_classification = os.environ['MODEL_CLASSIFICATION']
model_labeling = os.environ['MODEL_LABELING']

# create a CiRA converter object 
cira = CiRAConverter(classifier_causal_model_path=model_classification, converter_s2l_model_path=model_labeling)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer

In [3]:
# define an exemplary sentence
sentence: str = "If the red button is pressed and background operations are not running then the system shuts down."

In [4]:
# classify the sentence
causal,confidence = cira.classify(sentence)
print(f'CiRA classified the sentence "{sentence}" to be {"*" if causal else "*non"}causal* with a confidence of {confidence:.2%}.')

CiRA classified the sentence "If the red button is pressed and background operations are not running then the system shuts down." to be *causal* with a confidence of 98.47%.


In [6]:
# if the sentence is causal, process it by generating the labels, graph, and consequent test suite from the sentence
if causal:
    labels, graph, suite = cira.process(sentence)

    print(f'Labels associated to the sentence: {labels}\n')
    print(f'Converted cause-effect-graph: {graph}\n')
    print(f'Minimal test suite: \n{suite}')

Labels associated to the sentence: [[3> (L0) Cause1  (AND L1): [3> (L4) Variable <17] [18> (L7) Condition <28] <28], [33> (L1) Cause2  (None L2): [33> (L5) Variable <54] [63> (L8) Condition <70] [55> (L10) Negation <62] <70], [76> (L2) Effect1 : [76> (L6) Variable <86] [87> (L9) Condition <97] <97], [29> (L3) Conjunction <32], [3> (L4) Variable <17], [33> (L5) Variable <54], [76> (L6) Variable <86], [18> (L7) Condition <28], [63> (L8) Condition <70], [87> (L9) Condition <97], [55> (L10) Negation <62]]

Converted cause-effect-graph: ([the red button].(is pressed) && NOT [background operations].(running)) ===> [the system].(shuts down)

Minimal test suite: 
  id  | the red button    background operations    | the system
----  ------------------  -----------------------  ----------------
   1  | is pressed        not running              | shuts down
   2  | not is pressed    not running              | not shuts down
   3  | is pressed        running                  | not shuts down
